In [2]:
% matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import requests
import json
import datetime
import random

# Using IMDBpie API to get data from IMDB website
from imdbpie import Imdb

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer

In [3]:
# Create an instance of imdbpie

imdb = Imdb()
imdb = Imdb(anonymize=True) # to proxy requests

# Getting top 250 ever

top250 = imdb.top_250()

type(top250)


list

In [4]:
df250 = pd.DataFrame(top250)
df250.head(2)

,can_rate,image,num_votes,rating,tconst,title,type,year
0,True,{u'url': u'https://images-na.ssl-images-amazon...,1724776,9.3,tt0111161,The Shawshank Redemption,feature,1994
1,True,{u'url': u'https://images-na.ssl-images-amazon...,1178676,9.2,tt0068646,The Godfather,feature,1972


In [5]:
# Function to get movie entries using OMDB API, code obtained from lab

def get_entry(entry):
    res = requests.get('http://www.omdbapi.com/?i='+entry)
    if res.status_code != 200:
        print entry, res.status_code
    else:
        print '.',
    try:
        j = json.loads(res.text)
    except ValueError:
        j = None
    return j

# Getting a list of title code for top250 movies
entries = list(df250['tconst'].values)

In [6]:
entries_dict_list = [get_entry(e) for e in entries]

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .


In [7]:
df = pd.DataFrame(entries_dict_list)

In [9]:
df.head(1)

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,Released,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes
0,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Nominated for 7 Oscars. Another 18 wins & 30 n...,USA,Frank Darabont,"Crime, Drama",English,80,Two imprisoned men bond over a number of years...,https://images-na.ssl-images-amazon.com/images...,R,14 Oct 1994,True,142 min,The Shawshank Redemption,movie,"Stephen King (short story ""Rita Hayworth and S...",1994,tt0111161,9.3,"1,711,064"


In [10]:
df.shape

(250, 20)

In [15]:
# Reverting to webscrapping to get gross of each movie
# Creating a function to scrap gross, company credits and technical details of each movie

def get_gross(entry):
    response = requests.get('http://www.imdb.com/title/' + entry)
    html = response.text
    try:
        
        gross = re.findall('Gross:</h4>\s*\$([^ ]*)', html)
        return int(gross[0].replace(',',''))
    except Exception as ex:
        print ex, entry, requests.status_codes
        return None

In [16]:
gross_list = [(x,get_gross(x)) for x in entries]

list index out of range tt0050083 <module 'requests.status_codes' from '//anaconda/lib/python2.7/site-packages/requests/status_codes.pyc'>
list index out of range tt0038650 <module 'requests.status_codes' from '//anaconda/lib/python2.7/site-packages/requests/status_codes.pyc'>
list index out of range tt0064116 <module 'requests.status_codes' from '//anaconda/lib/python2.7/site-packages/requests/status_codes.pyc'>
list index out of range tt0034583 <module 'requests.status_codes' from '//anaconda/lib/python2.7/site-packages/requests/status_codes.pyc'>
list index out of range tt0021749 <module 'requests.status_codes' from '//anaconda/lib/python2.7/site-packages/requests/status_codes.pyc'>
list index out of range tt0057012 <module 'requests.status_codes' from '//anaconda/lib/python2.7/site-packages/requests/status_codes.pyc'>
list index out of range tt0043014 <module 'requests.status_codes' from '//anaconda/lib/python2.7/site-packages/requests/status_codes.pyc'>
list index out of range tt0

In [17]:
df_gross = pd.DataFrame(gross_list, columns = ['imdbID','Gross'])
df_gross.head(2)

,imdbID,Gross
0,tt0111161,28341469.0
1,tt0068646,134821952.0


In [18]:
# function to get technical details of movie

def get_sound(entry):
    response = requests.get('http://www.imdb.com/title/' + entry)
    html = response.text
    try:
        sound_list = re.findall('sound_mixes=([^&]*)', html)
        sound = ''
        for i in sound_list:
            sound = sound + ' ' + i
        return sound
        
    except Exception as ex:
        print entry, ex, requests.status_codes
        return None

sound_list = [(x, get_sound(x)) for x in entries]

In [22]:
df_sound = pd.DataFrame(sound_list, columns = ['imdbID','sound'])
df_sound.head(2)

,imdbID,sound
0,tt0111161,dolby_digital
1,tt0068646,dts mono


In [19]:
# Deleting Poster and Response columns
del df['Poster']
del df['Response']

# Replace "N/A" with ''
df.loc[:,'Awards'] = df['Awards'].apply(lambda x: (x).replace("N/A", ''))

# Converting runtime to integer of minutes
df.loc[:,'Runtime'] = df['Runtime'].apply(lambda x: int(x.split()[0]))

# Converting ratings to float
df.loc[:, 'imdbRating'] = df['imdbRating'].astype(float)

# Converting votes to int
df.loc[:, 'imdbVotes'] = df['imdbVotes'].apply(lambda x: int(x.replace(',','')))

# Replace "N/A" with np.nan
df.loc[:,'Metascore'] = df['Metascore'].apply(lambda x: (x).replace("N/A", '0'))
df.loc[:,'Metascore'] = df['Metascore'].astype(float)
df.loc[:,'Metascore'] = df['Metascore'].apply(lambda x: np.nan if x == 0 else x)


def change_date(x):
    try:
        return datetime.datetime.strptime(x, "%d %b %Y")
    except:
        return None
df.loc[:,'Released'] = df['Released'].apply(change_date)

df.loc[:,'Year'] = df["Year"].astype(int)

In [20]:
df = pd.merge(df, df_gross, on= 'imdbID')
df.head(2)

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Rated,Released,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes,Gross
0,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Nominated for 7 Oscars. Another 18 wins & 30 n...,USA,Frank Darabont,"Crime, Drama",English,80.0,Two imprisoned men bond over a number of years...,R,1994-10-14,142,The Shawshank Redemption,movie,"Stephen King (short story ""Rita Hayworth and S...",1994,tt0111161,9.3,1711064,28341469.0
1,"Marlon Brando, Al Pacino, James Caan, Richard ...",Won 3 Oscars. Another 23 wins & 27 nominations.,USA,Francis Ford Coppola,"Crime, Drama","English, Italian, Latin",100.0,The aging patriarch of an organized crime dyna...,R,1972-03-24,175,The Godfather,movie,"Mario Puzo (screenplay), Francis Ford Coppola ...",1972,tt0068646,9.2,1177812,134821952.0


In [23]:
df = pd.merge(df, df_sound, on= 'imdbID')
df.head(2)

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Rated,Released,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes,Gross,sound
0,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Nominated for 7 Oscars. Another 18 wins & 30 n...,USA,Frank Darabont,"Crime, Drama",English,80.0,Two imprisoned men bond over a number of years...,R,1994-10-14,142,The Shawshank Redemption,movie,"Stephen King (short story ""Rita Hayworth and S...",1994,tt0111161,9.3,1711064,28341469.0,dolby_digital
1,"Marlon Brando, Al Pacino, James Caan, Richard ...",Won 3 Oscars. Another 23 wins & 27 nominations.,USA,Francis Ford Coppola,"Crime, Drama","English, Italian, Latin",100.0,The aging patriarch of an organized crime dyna...,R,1972-03-24,175,The Godfather,movie,"Mario Puzo (screenplay), Francis Ford Coppola ...",1972,tt0068646,9.2,1177812,134821952.0,dts mono


In [24]:
# Merging actors' name so we can countvectorizer FirstLast name together
df.loc[:, 'Actors'] = df['Actors'].apply(lambda x: x.replace('-',''))
df.loc[:, 'Director'] = df['Director'].apply(lambda x: x.replace(' ',''))
df.loc[:, 'Genre'] = df['Genre'].apply(lambda x: x.replace(' ',''))
df.loc[:, 'Writer'] = df['Writer'].apply(lambda x: x.replace(' ',''))
df.loc[:, 'Country'] = df['Country'].apply(lambda x: x.replace(' ',''))
df.loc[:, 'sound'] = df['sound'].apply(lambda x: x.replace('_',''))

In [25]:
def win_oscar(x):
    try:
        return int(re.findall('Won\s(\d)\sOscars',x)[0])
    except:
        return 0
df['win_oscar'] = df['Awards'].apply(win_oscar)


def nom_oscar(x):
    try:
        return int(re.findall('Nominated\sfor\s(\d)\sOscar',x)[0])
    except:
        return 0
df['nom_oscar'] = df['Awards'].apply(nom_oscar)

def wins(x):
    try:
        return int(re.findall('(\d+)\swin',x)[0])
    except:
        return 0
df['awards_won'] = df['Awards'].apply(wins)

def nom(x):
    try:
        return int(re.findall('(\d+)\snomination',x)[0])
    except:
        return 0
df['nominations'] = df['Awards'].apply(nom)


In [27]:
# Generic vectorizer

v = CountVectorizer(
    binary=True,  # Create binary features
    stop_words='english', # Ignore common words such as 'the', 'and'
    max_features=50, # Only use the top 50 most common words
)

In [29]:
# This builds a matrix with a row per website (or data point) and column per word (using all words in the dataset)
actors = v.fit_transform(df.Actors).todense()
col = v.get_feature_names()
# To differentiate key words
col_n = []

for i in col:
    j = "actor_"+ i    
    col_n.append(j)

actors = pd.DataFrame(actors, columns= col_n)
actors_df = pd.DataFrame(actors.sum())
actors_df.columns = ['Count']
actors_df.sort_values(['Count'],inplace = True)

In [55]:
actors.shape

(250, 50)

In [34]:
#Director
direct = v.fit_transform(df.Director).todense()
col = v.get_feature_names()
col_n = []
for i in col:
    j = "dir_"+ i    
    col_n.append(j)
direct = pd.DataFrame(direct, columns= col_n)
direct_df = pd.DataFrame(direct.sum())
direct_df.columns = ['Count']
direct_df.sort_values(['Count'],inplace = True)

In [35]:
#Genre
genre = v.fit_transform(df.Genre).todense()
col = v.get_feature_names()
col_n = []
for i in col:
    j = "genre_"+ i    
    col_n.append(j)
genre = pd.DataFrame(genre, columns= col_n)

In [36]:
#Language
lang = v.fit_transform(df.Language).todense()
col = v.get_feature_names()
lang = pd.DataFrame(lang, columns= col)

In [37]:
#Plot
plot = v.fit_transform(df.Plot).todense()
col = v.get_feature_names()
col_n = []
for i in col:
    j = "plot_"+ i    
    col_n.append(j)
plot = pd.DataFrame(plot, columns= col_n)
plot_df = pd.DataFrame(plot.sum())
plot_df.columns = ['Count']
plot_df.sort_values(['Count'],inplace = True)

In [38]:
#Title
title = v.fit_transform(df.Title).todense()
col = v.get_feature_names()
col_n = []
for i in col:
    j = "title_"+ i    
    col_n.append(j)
title = pd.DataFrame(title, columns= col_n)
title_df = pd.DataFrame(title.sum())
title_df.columns = ['Count']
title_df.sort_values(['Count'],inplace = True)

In [39]:
#Country
country = v.fit_transform(df.Country).todense()
col = v.get_feature_names()
country = pd.DataFrame(country, columns= col)
country = v.fit_transform(df.Country).todense()
col = v.get_feature_names()
country = pd.DataFrame(country, columns= col)

In [40]:
#Sound
sound = v.fit_transform(df.sound).todense()
col = v.get_feature_names()
sound = pd.DataFrame(sound, columns= col)
sound_df = pd.DataFrame(sound.sum())
sound_df.columns = ['Count']
sound_df.sort_values(['Count'],inplace = True)

In [42]:
df_rate = pd.get_dummies(df['Rated'])

#Dropping "N/A" column
del df_rate['N/A']

In [43]:
df.loc[:, ['nom_oscar', 'awards_won']] = df[['nom_oscar', 'awards_won']].astype(int)

In [44]:
a = df['Gross'].median()
b = df['Gross'].mad()
df.loc[:, 'Gross'] = df['Gross'].apply(lambda x:x if x > 0 else abs(a + (np.random.normal() * b)))

In [45]:
a = df['Metascore'].mean()
b = df['Metascore'].std()
df.loc[:, 'Metascore'] = df['Metascore'].apply(lambda x:x if x > 0 else abs(a + (np.random.normal() * b)))

In [46]:
want_list = ['Metascore', 'Year', 'Runtime', \
             'imdbVotes', 'Gross', 'win_oscar', \
             'nom_oscar', 'awards_won', 'nominations']

scaler = StandardScaler()
df.loc[:, want_list] = scaler.fit_transform(df[want_list])

In [47]:
y = df['imdbRating']
X = pd.concat([df[want_list], actors, direct, genre, lang, plot, title, country, sound, df_rate], axis=1)
X.shape

(250, 351)

In [49]:
from sklearn.cross_validation import cross_val_score, KFold, cross_val_predict

from sklearn.grid_search import GridSearchCV

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor

//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [60]:
rand_state = 4
model_list = [('DecisionTreeRegressor',DecisionTreeRegressor(random_state=rand_state)),
    ('RandomForestRegressor', RandomForestRegressor(random_state=rand_state)),
    ('ExtraTreesRegressor', ExtraTreesRegressor(random_state=rand_state)),
    ('GradientBoostingRegressor', GradientBoostingRegressor(random_state=rand_state)),
    ('AdaBoostRegressor', AdaBoostRegressor(random_state=rand_state))]

In [65]:
def plain_model(clf):
    score = cross_val_score(clf, X, y, scoring='neg_mean_squared_error', cv = 10)
    return score

model_performance = []

# for i in model_list:
#     result = plain_model(i[1])
#     model_performance.append([i[0], abs(result.mean()), result.std()])

In [66]:
plain_model(model_list[4][1])

array([-0.25023485, -0.05701261, -0.01958868, -0.0222512 , -0.00566137,
       -0.01369055, -0.02123456, -0.03289753, -0.04254678, -0.04877537])

In [ ]:
cross_val_score